In [2]:
import pdfplumber
import json
import psycopg2
import requests


In [3]:
# 📌 EXTRACTION DU TEXTE PDF
def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = "\n".join(page.extract_text() for page in pdf.pages if page.extract_text())
    return text

In [13]:
texte = extract_text_from_pdf("C:/Users/danie/OneDrive/Documents/GitHub/Challenge_web_mining/data/lm/homme7lm.pdf")
print(texte)

Objet : Candidature au poste de Data Scientist
Madame, Monsieur,
Actuellement en Licence 3 Informatique, parcours MIAGE, je suis passionné par la science des
données et son application dans la prise de décision stratégique. C'est dans cette optique que je
souhaite intégrer votre entreprise en tant que Data Scientist.
Grâce à ma formation et à mes expériences en analyse et traitement de données, j'ai développé
une expertise en machine learning, en visualisation et en manipulation des données à l'aide de
Python, SQL et Power BI. Mon parcours m'a permis de travailler sur des projets d'analyse prédictive
et d'extraction de tendances à partir de grandes volumétries de données.
Curieux, méthodique et rigoureux, je suis motivé à intégrer une équipe dynamique où je pourrai
mettre à profit mes compétences et apprendre de nouvelles techniques d'analyse. Je reste
disponible pour un entretien afin de discuter plus en détail de ma candidature.
Dans l'attente de votre retour, veuillez agréer, Madame

In [4]:
# 📌 APPEL API MISTRAL AVEC JSON STRICT
MISTRAL_API_KEY = "Yp0Uo7Vx4uSJIlc94dj3MA5ME71KpwIR"
API_URL = "https://api.mistral.ai/v1/chat/completions"

def query_mistral(prompt):
    headers = {
        "Authorization": f"Bearer {MISTRAL_API_KEY}",
        "Content-Type": "application/json"
    }

    data = {
        "model": "mistral-medium",
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.7
    }

    response = requests.post(API_URL, headers=headers, json=data)

    if response.status_code == 200:
        try:
            return json.loads(response.json()["choices"][0]["message"]["content"])
        except json.JSONDecodeError:
            print("⚠️ Erreur : réponse Mistral non valide en JSON.")
            return None
    else:
        print("❌ Erreur API :", response.text)
        return None

# 📌 EXTRACTION DES DONNÉES AVEC FORMAT JSON STRICT
def extract_data(text):
    prompt = f"""
    Voici une lettre de motivation :
    "{text}"

    Formate ta réponse en JSON avec ces champs :
    {{
      "competences": "compétence1", "compétence2",
      "motivations": "motivation1", "motivation2",
      "lieu": "ville"
    }}
    """
    return query_mistral(prompt)

In [14]:
data = extract_data(texte)
print(data)

⚠️ Erreur : réponse Mistral non valide en JSON.
None


In [5]:

# 📌 INSÉRER DANS POSTGRESQL
def insert_into_db(data):
    if not data:
        print("⚠️ Pas de données à insérer.")
        return

    try:
        conn = psycopg2.connect(
            dbname="webmining",
            user="postgres",
            password="postgres",
            host="localhost",
            port="5432",
        )
        cursor = conn.cursor()
        

        cursor.execute("""
            CREATE TABLE IF NOT EXISTS lm (
                id SERIAL PRIMARY KEY,
                competences JSON,
                motivations JSON,
                lieu TEXT
            )
        """)

        cursor.execute("""
            INSERT INTO lm (competences, motivations, lieu)
            VALUES (%s, %s, %s)
        """, (json.dumps(data["competences"]), json.dumps(data["motivations"]), data["lieu"]))

        conn.commit()
        print("✅ Données insérées avec succès !")

    except Exception as e:
        print("❌ Erreur PostgreSQL :", e)

    finally:
        cursor.close()
        conn.close()



In [9]:
# 📌 EXÉCUTION DU PIPELINE
if data:
    print("✅ Compétences :", data["competences"])
    print("✅ Motivations :", data["motivations"])
    print("✅ Lieu :", data["lieu"])
    insert_into_db(data)

✅ Compétences : ['analyse de données', 'modélisation de machine learning', 'IA générative', 'Python', 'SQL', 'Power BI', 'Tableau']
✅ Motivations : ['évoluer au sein d’un acteur majeur des télécommunications', 'mettre mes connaissances en data science au service de vos objectifs stratégiques']
✅ Lieu : Lyon
✅ Données insérées avec succès !


In [9]:
#affichage des tables dans la bdd
import pandas as pd

conn = psycopg2.connect(
            dbname="webmining",
            user="postgres",
            password="postgres",
            host="localhost",
            port="5432",
        )
cur = conn.cursor()
query = """
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public';
"""

cur.execute(query)
tables = cur.fetchall()

# Afficher les tables sous forme de DataFrame
df_tables = pd.DataFrame(tables, columns=["Table Name"])
print(df_tables)

            Table Name
0                   lm
1             annonces
2  annonce_competences
3          competences


C:\Users\danie\AppData\Local\Temp\ipykernel_33164\3679823431.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


### embedding


In [12]:
from sentence_transformers import SentenceTransformer

def get_competence_embeddings(competences):

    # Charger le modèle
    model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
    
    # Obtenir les embeddings
    embeddings = model.encode(competences)
    
    # Retourner un dictionnaire des embeddings
    return {competence: embedding for competence, embedding in zip(competences, embeddings)}


c:\Users\danie\anaconda3\envs\ocrcvlm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:

embeddings_dict = get_competence_embeddings(data["competences"])
for competence, embedding in embeddings_dict.items():
    print(f"Embeddings de '{competence}': {embedding}")

Embeddings de 'analyse de données': [-3.91415656e-01  7.02954173e-01  2.10297629e-01 -1.87427133e-01
 -1.12183154e-01  2.63784286e-02  7.23191440e-01 -7.06837475e-02
  1.76174894e-01 -8.40033069e-02  1.61910683e-01 -5.82956433e-01
  6.20934293e-02 -2.14757308e-01  1.38983771e-01 -1.22914433e-01
  7.06192479e-03 -9.21110511e-02 -3.08067724e-02  3.16265792e-01
  1.04365617e-01  5.82234263e-01  8.21356848e-03 -4.69451904e-01
  2.98324049e-01 -2.18996964e-02  6.63493015e-03  6.19996712e-02
  5.71180582e-02 -2.44487047e-01  7.66216516e-02  2.12265715e-01
 -3.64372432e-01  4.30683434e-01 -2.92696506e-01 -4.27508920e-01
 -1.54161334e-01 -2.56205142e-01  3.16251665e-01  9.45896566e-01
 -5.73670983e-01  4.20899391e-01 -2.00253561e-01 -1.44261122e-01
  6.44158125e-02  3.42894495e-01 -2.51067340e-01  4.38909054e-01
 -5.44468760e-02 -9.15521011e-02 -2.42464602e-01 -2.86676407e-01
 -2.52819508e-01 -2.61771709e-01  5.67613721e-01 -7.20457733e-01
 -3.36357772e-01 -2.35240132e-01 -4.25381124e-01  2.03

In [7]:
# 📌 Fonction pour exécuter tout
def process_and_store_lm(lm_pdf_path):
    # Extraire le texte de la LM
    text = extract_text_from_pdf(lm_pdf_path)

    # Analyser les données avec Mistral
    result = extract_data(text)
    
    # Insérer les données extraites dans la base de données
    if result:
        insert_into_db(result)
        return result
    else:
        return None

In [11]:
import os

# Dossier contenant les PDFs
pdf_folder = "../data/lm"

# Récupérer tous les fichiers PDF du dossier
pdf_files = [f for f in os.listdir(pdf_folder) if f.endswith(".pdf")]

# Boucler sur chaque PDF et exécuter la fonction
for pdf in pdf_files:
    pdf_path = os.path.join(pdf_folder, pdf)
    result = process_and_store_lm(pdf_path)
    if result:
        print(f"✅ Traitement réussi pour {pdf}")
    else:
        print(f"❌ Échec du traitement pour {pdf}")


⚠️ Erreur : réponse Mistral non valide en JSON.
❌ Échec du traitement pour femme10lm.pdf
⚠️ Erreur : réponse Mistral non valide en JSON.
❌ Échec du traitement pour femme1lm.pdf
⚠️ Erreur : réponse Mistral non valide en JSON.
❌ Échec du traitement pour femme2lm.pdf
✅ Données insérées avec succès !
✅ Traitement réussi pour femme3lm.pdf
⚠️ Erreur : réponse Mistral non valide en JSON.
❌ Échec du traitement pour femme4lm.pdf
✅ Données insérées avec succès !
✅ Traitement réussi pour femme5lm.pdf
✅ Données insérées avec succès !
✅ Traitement réussi pour femme6lm.pdf
✅ Données insérées avec succès !
✅ Traitement réussi pour femme7lm.pdf
⚠️ Erreur : réponse Mistral non valide en JSON.
❌ Échec du traitement pour femme8lm.pdf
✅ Données insérées avec succès !
✅ Traitement réussi pour femme9lm.pdf
✅ Données insérées avec succès !
✅ Traitement réussi pour homme10lm.pdf
⚠️ Erreur : réponse Mistral non valide en JSON.
❌ Échec du traitement pour homme1lm.pdf
⚠️ Erreur : réponse Mistral non valide en JS

In [10]:
# Nom de la table à afficher
table_name = "annonces"

# Exécuter la requête
query = f"SELECT * FROM {table_name} LIMIT 5;"
df = pd.read_sql(query, conn)

# Afficher le DataFrame
print(df)


  reference                                     intitule_poste  \
0   0907189                           Développeur Big Data H/F   
1   1286647          Développeur / Développeuse Big Data (H/F)   
2   1480816                    Data Scientist - Alternance H/F   
3   186RJZQ                                Data Engineer (H/F)   
4   187NTTM  Développeur/Développeuse Python et Data Engine...   

                                         description  \
0  POSTE : Développeur Big Data H/F\nDESCRIPTION ...   
1  Nous recherchons un Data Engineer Sénior pour ...   
2  POSTE : Data Scientist - Alternance H/F\nDESCR...   
3  Weldom, une rencontre évidente\nDes défis à re...   
4  Vous avez une expérience en matière Python et ...   

                     experience                                       divers  
0             Expérience exigée                                               
1  Expérience exigée de 5 An(s)                                               
2             Expérience exig

C:\Users\danie\AppData\Local\Temp\ipykernel_31576\941641511.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [ ]:
# Nom de la table à afficher
table_name = "lm"

# Exécuter la requête
query = f"SELECT * FROM {table_name} LIMIT 5;"
df = pd.read_sql(query, conn)

# Afficher le DataFrame
print(df)

In [ ]:
conn = psycopg2.connect(
            dbname="webmining",
            user="postgres",
            password="postgres",
            host="localhost",
            port="5432",
        )

query = "SELECT * FROM lm;"


rows = cursor.fetchall()


for row in rows:
    print(row)